In [9]:
import time
import pandas as pd
from census import Census

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

# Conditions

What are the conditions of the census tracts within Richmond City, North Richmond, and Rollingwood, California, in 2022 and 2017?

## Race
What is the racial and ethnic percentages of each census tract? Are there any census tracts with MOEs that are too high to include?

In [7]:
def process_acs_df(in_df):
    '''This is a *docstring*. It's a brief explanation of what the function does,
    and what inputs and outputs it has.

    Inputs:
    - in_df (pd.DataFrame): a DataFrame containing race/ethnicity data from Table B03002

    Outputs:
    A modified version of in_df with certain races grouped and proportions and margins
    of error calculated
    '''
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df = in_df.copy()

    ### CLEAN UNUSUAL MOES
    df = df.replace(-555555555, 0)

    ### AGGREGATE ESTIMATES
    # Define a list of columns to combine into "nh_other"
    nh_other_cols = ['nh_native', 'nh_pi', 'nh_1other', 'nh_multi']

    # Calculate nh_other
    df['nh_other'] = df[nh_other_cols].sum(axis='columns')

    # Use a list comprehension to append "_moe" to all strings in our list
    nh_other_moes = [f'{col}_moe' for col in nh_other_cols]

    # Use this new list to calculate nh_other_moe    
    df['nh_other_moe'] = (df[nh_other_moes]**2).sum(axis='columns')**0.5

    ### CALCULATE PROPORTIONS
    # Iterate through the racial/ethnic groups we're using
    for group in ['nh_white', 'nh_black', 'nh_asian', 'nh_other', 'hispanic']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

    return df  # Don't forget to *return* the modified DataFrame

In [39]:
# Define the dict of variables to pull and rename
variables_of_interest = {
    'NAME': 'NAME',
    # 'GEO_ID': 'GEO_ID',
    'B03002_001E': 'total',
    'B03002_001M': 'total_moe',
    'B03002_003E': 'nh_white',
    'B03002_003M': 'nh_white_moe',
    'B03002_004E': 'nh_black',
    'B03002_004M': 'nh_black_moe',
    'B03002_005E': 'nh_native',
    'B03002_005M': 'nh_native_moe',
    'B03002_006E': 'nh_asian',
    'B03002_006M': 'nh_asian_moe',
    'B03002_007E': 'nh_pi',
    'B03002_007M': 'nh_pi_moe',
    'B03002_008E': 'nh_1other',
    'B03002_008M': 'nh_1other_moe',
    'B03002_009E': 'nh_multi',
    'B03002_009M': 'nh_multi_moe',
    'B03002_012E': 'hispanic',
    'B03002_012M': 'hispanic_moe',
}

# Get ACS 2022 5 year Table B03002 in Richmond City, North Richmond, Rollingwood, CA
# Place:60620,52162,62700 TBD: East Richmond heights is place 21061
# county:013
c_tracts = 'tract:380001,380002,378000,365002,365003,379000,377000,376000,374000,371000,367200'

df_tracts = pd.DataFrame(
    c.acs5.get(
        list(variables_of_interest.keys()),
        {'for': c_tracts, 'in': 'state:06 county:013'},
        year=2022
    )
)

# Rename the DataFrame columns again using the dict
df_tracts = df_tracts.rename(columns=variables_of_interest)

df_tracts
# process_acs_df(df_tracts)

,NAME,total,total_moe,nh_white,nh_white_moe,nh_black,nh_black_moe,nh_native,nh_native_moe,nh_asian,...,nh_pi_moe,nh_1other,nh_1other_moe,nh_multi,nh_multi_moe,hispanic,hispanic_moe,state,county,tract
0,Census Tract 3650.02; Contra Costa County; Cal...,5171.0,619.0,261.0,168.0,779.0,291.0,0.0,19.0,353.0,...,243.0,0.0,19.0,186.0,132.0,3347.0,596.0,06,013,365002
1,Census Tract 3650.03; Contra Costa County; Cal...,5416.0,838.0,1035.0,441.0,1576.0,441.0,35.0,59.0,867.0,...,49.0,361.0,382.0,214.0,145.0,1300.0,458.0,06,013,365003
2,Census Tract 3672; Contra Costa County; Califo...,5869.0,760.0,292.0,100.0,755.0,341.0,0.0,19.0,1366.0,...,72.0,46.0,85.0,205.0,121.0,3158.0,661.0,06,013,367200
3,Census Tract 3710; Contra Costa County; Califo...,5776.0,693.0,1543.0,315.0,590.0,229.0,0.0,19.0,888.0,...,19.0,0.0,19.0,481.0,322.0,2274.0,564.0,06,013,371000
4,Census Tract 3740; Contra Costa County; Califo...,4687.0,717.0,888.0,232.0,497.0,159.0,0.0,13.0,522.0,...,13.0,29.0,52.0,116.0,92.0,2635.0,685.0,06,013,374000
5,Census Tract 3760; Contra Costa County; Califo...,6712.0,1000.0,41.0,32.0,888.0,273.0,21.0,37.0,286.0,...,19.0,0.0,19.0,316.0,166.0,5160.0,1005.0,06,013,376000
6,Census Tract 3770; Contra Costa County; Califo...,7826.0,1278.0,366.0,206.0,1720.0,709.0,0.0,19.0,509.0,...,19.0,36.0,63.0,197.0,191.0,4998.0,1316.0,06,013,377000
7,Census Tract 3780; Contra Costa County; Califo...,3887.0,532.0,2548.0,439.0,39.0,36.0,0.0,13.0,308.0,...,13.0,28.0,42.0,249.0,145.0,715.0,249.0,06,013,378000
8,Census Tract 3790; Contra Costa County; Califo...,8383.0,1224.0,177.0,73.0,1509.0,382.0,0.0,19.0,173.0,...,108.0,131.0,197.0,325.0,211.0,5972.0,1158.0,06,013,379000
9,Census Tract 3800.01; Contra Costa County; Cal...,2896.0,355.0,1196.0,207.0,543.0,299.0,5.0,9.0,582.0,...,13.0,0.0,13.0,214.0,134.0,356.0,246.0,06,013,380001
